<a href="https://colab.research.google.com/github/CyrilZzz/nlp_project/blob/Alex-test/projet_nlp_3A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import gdown

url = 'https://drive.google.com/uc?id=1eNJn7rX2OE9UKrrknmvClt3IK2cUTRjB&export=download&confirm=t'
output = 'dataset.tar.xz'

gdown.download(url, output, quiet=False)



Downloading...
From: https://drive.google.com/uc?id=1eNJn7rX2OE9UKrrknmvClt3IK2cUTRjB&export=download&confirm=t
To: /content/20150428_collected_images.tgz
100%|██████████| 334M/334M [00:06<00:00, 54.7MB/s]


'20150428_collected_images.tgz'

In [7]:
import tarfile

with tarfile.open('dataset.tar.xz') as f:
    f.extractall('.')

In [42]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.1 MB/s eta 0:00:00


In [45]:
df = pd.read_csv('/content/original/imdb/roberta/textfooler/labels/roberta-base-imdb_textfooler_pos.csv')

In [55]:
df["perturbed_text"][9999]

"Melancholy of Haruhi Suzumiya is based on somewhat [[even]] formula by now of Japanese school drama anime. The formula somewhat goes like this:<br /><br />1. The main protagonist comes from world outside the normal society, and has super powers.<br /><br />2. There's a very beautiful and sexy girl in a supporting role.<br /><br />3. A normal character is in there who shares the main role in the story.<br /><br />4. Unusual things happens in an usual social settings.<br /><br />5. Sometimes the story is about the main protagonist, and the normal character that connects to the existence or destruction of the world.<br /><br />6. Absolutely no effort is spent by anyone to gain all the magical powers. They just have it.<br /><br />7. Usually, people outside of this tight nit group is not aware of their super powers, and goes on with their daily lives.<br /><br />So there you have it. [[Gloomy]] of Suzumiya Haruhi is made along these lines which became the success formula for comics and an

In [ ]:
######## code random dessous

In [ ]:
import re
from spacy.lang.en import English

spacy_tokenizer = English().tokenizer

def clean_str(string, tokenizer=spacy_tokenizer):
    """
    Parts adapted from https://github.com/Shawn1993/cnn-text-classification-pytorch/blob/master/mydatasets.py
    """
    assert isinstance(string, str)
    string = string.replace("<br />", "")
    string = re.sub(r"[^a-zA-Z0-9.]+", " ", string)

    return (
        string.strip().lower().split()
        if tokenizer is None
        else [t.text.lower() for t in tokenizer(string.strip())]
    )

def pad(max_len, seq, token):
	assert isinstance(seq, list)
	abs_len = len(seq)
	if abs_len > max_len:
		seq = seq[:max_len]
	else:
		seq += [token] * (max_len - abs_len)
	return seq

def fgws_preprocess(sentence):
	sentence = clean_str(sentence)
	sentence = pad(200, sentence, "<pad>")
	sentence = ' '.join(sentence)
	return sentence

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.5 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset

# Define the dataset class
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenize the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

data_path = 

# Load the data
train_texts = [...] # list of training texts
train_labels = [...] # list of training labels
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_len=128)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
loss_fn = nn.CrossEntropyLoss()

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        optimizer.zero_grad()
    

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

TypeError: ignored

In [ ]:
class TweetModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, pretrained_vectors=None):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TweetModel, self).__init__()

        # apply the pretrained embeddings to transform our token indices, into vectors
        # we set freeze to True in order not to continue modifying the pretrained embeddings
        self.ebd = torch.nn.Embedding.from_pretrained(pretrained_vectors, freeze=True)

        # create two linear layers https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
        # the hidden layer should have the hidden_dim as input and ouput dimensions
        self.hidden_linear_layer = torch.nn.Linear(hidden_dim, hidden_dim, bias=True) # CODE ME
        # the classification layer should use the number of classes as output_dim
        self.classification_layer = torch.nn.Linear(hidden_dim, output_dim, bias=True)# CODE ME
        
        # softmax layer to compute class probabilities
        # https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html?highlight=softmax#torch.nn.Softmax
        self.softmax = torch.nn.Softmax()# CODE ME

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        # apply the pretrained embeddings
        x  = self.ebd(x)

        # compute the mean of x along the 1 axis
        x = x.mean(dim=1)# CODE ME
        
        h  = torch.relu(self.hidden_linear_layer( x ))
        
        h  = self.classification_layer(x)
        
        # apply the softmax layer
        logits = self.softmax(h)
        return logits

## <font color="chillipepper">What about BERT?</font>

You can find a tutorial on finetuning here : https://huggingface.co/docs/transformers/training

And look at the following code cells which modify the model you used to exploit BERT pretrained vectors and finetune them.

We consider `bert-tiny`to have a very small version of BERT (so the finetuning is faster), this renders the results less viable. To finetune the real `bert-base-cased` with all the training samples would take a lot of time.

In [ ]:
tweet_eval = load_dataset('tweet_eval', 'emoji')
def adjust_labels(entries):
  res = {}
  res['text'] = entries['text']
  res['label'] = [ label - 1 for label in entries['label']]
  return res

for split in ['train', 'validation', 'test']:
  rows_to_select = [ i for i, entry in enumerate(tweet_eval[split]) if entry['label'] <= 3 and entry['label'] != 0 ]
  tweet_eval[split] = tweet_eval[split].select(rows_to_select)
  tweet_eval[split] = tweet_eval[split].map(lambda e: adjust_labels(e)  , batched=True)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=True)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = tweet_eval.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(200))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [ ]:
from transformers import AutoModelForSequenceClassification
# This model is equal to BERT + a linear layer for classification. In our custom model we designed a FastText + a hidden layer and linear layer for classification
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=3)

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
from datasets import load_metric
metric = load_metric("accuracy")
model.eval()
preds, trues = [], []
for i, batch in tqdm(enumerate(eval_dataloader), desc="evaluating", total=eval_dataloader.__len__()):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

    _, tag_seq  = torch.max(logits, 1)
    preds.extend(tag_seq.cpu().detach().tolist())
    trues.extend(batch['labels'].cpu().detach().tolist())

metric.compute()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, plot_roc_curve
print(classification_report(np.array(trues).flatten(), np.array(preds).flatten(), target_names=names))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, plot_roc_curve
import matplotlib.pyplot as plt
import seaborn as sn

cm = confusion_matrix(np.array(trues).flatten(), np.array(preds).flatten())
df_cm = pd.DataFrame(cm, index=names, columns=names)
# config plot sizes
sn.set(font_scale=1)
sn.heatmap(df_cm, annot=True, annot_kws={"size": 8}, cmap='coolwarm', linewidth=0.5, fmt="")
plt.show()

In [ ]:
import re
from spacy.lang.en import English

spacy_tokenizer = English().tokenizer

def clean_str(string, tokenizer=spacy_tokenizer):
    """
    Parts adapted from https://github.com/Shawn1993/cnn-text-classification-pytorch/blob/master/mydatasets.py
    """
    assert isinstance(string, str)
    string = string.replace("<br />", "")
    string = re.sub(r"[^a-zA-Z0-9.]+", " ", string)

    return (
        string.strip().lower().split()
        if tokenizer is None
        else [t.text.lower() for t in tokenizer(string.strip())]
    )

def pad(max_len, seq, token):
	assert isinstance(seq, list)
	abs_len = len(seq)
	if abs_len > max_len:
		seq = seq[:max_len]
	else:
		seq += [token] * (max_len - abs_len)
	return seq

def fgws_preprocess(sentence):
	sentence = clean_str(sentence)
	sentence = pad(200, sentence, "<pad>")
	sentence = ' '.join(sentence)
	return sentence